# Pandas - Agrupando datos
Inteligencia Artificial - Facundo A. Lucianna - CEIA - FIUBA

Una pieza esencial del análisis de datos es el agregado eficiente: calcular operaciones como `sum()`, `mean()`, `median()`, `min()` y `max()`. En esta notebook, exploraremos las agregaciones en Pandas, desde operaciones simples, hasta operaciones más sofisticadas basadas en el concepto de `groupby`.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

Para poder aplicar las herramientas de Pandas en este notebook usaremos el dataset Planets disponible a través del paquete Seaborn. Este conjunto proporciona información sobre planetas que los astrónomos han descubierto alrededor de otras estrellas (conocidos como exoplanetas):

In [2]:
planets = sns.load_dataset('planets')
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


## Agregaciones simples en Pandas

Al igual que con un array unidimensional de NumPy, para una `Serie` de Pandas, podemos hacer agregaciones que devuelven un único valor:

In [3]:
rng = np.random.RandomState(42)
random_numpy = rng.rand(5)
random_serie = pd.Series(rng.rand(5))
random_serie

0    0.155995
1    0.058084
2    0.866176
3    0.601115
4    0.708073
dtype: float64

In [4]:
random_serie.sum()

2.389441867818592

In [5]:
random_numpy.sum()

2.811925491708157

In [6]:
random_serie.mean()

0.4778883735637184

Hay un método conveniente llamado `describe()` que calcula varias agregaciones comunes para cada columna de un `DataFrame`. Vamos a usar esto en los datos de `planets`, eliminando por ahora las filas con valores faltantes:

In [7]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


Esto puede ser una forma útil de comenzar a entender las propiedades generales de un conjunto de datos, antes que empecemos a trabajar con nuestro modelo. 

*Por ejemplo, observamos en la columna de año que aunque los exoplanetas fueron descubiertos por primera vez en 1989, la mitad de todos los exoplanetas conocidos no fueron descubiertos hasta 2010 o después**.

----
## Group by
Las agregaciones simples pueden dar una idea del conjunto de datos, pero a menudo se prefiere agregar condicionalmente según alguna etiqueta o índice: esto se implementa en la llamada operación `groupby`. El nombre **group by** proviene de un comando en el lenguaje de base de datos SQL.

Lo que hace el `groupby` es:

1. Dividir y agrupar un DataFrame según el valor de una llave especificada.
2. Luego se calcula alguna función, generalmente una agregación, transformación o filtrado, dentro de los grupos individuales.
3. Al final se fusiona los resultados de estas operaciones en un array de salida.

Esto se puede realizar individualmente paso a paso, pero la función `groupby` nos facilita esta tarea.

Quizás las operaciones más importantes disponibles mediante `groupby` son `aggregate`, `filter`, `transform` y `apply`.

### Indexación de columnas

`groupby` admite la indexación de columnas de la misma manera que el `DataFrame`y devuelve un objeto GroupBy modificado:

In [8]:
planets.groupby('method')

In [9]:
planets.groupby('method')['orbital_period']

Acá seleccionamos un grupo particular del `DataFrame` original agrupándolo por referencia a su nombre de columna. Al igual que con el objeto `DataFrameGroupBy`, no se realiza ninguna computación hasta que llamemos a alguna agregación en el objeto:

In [10]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

### aggregate

El método `aggregate()` permite más flexibilidad. Puede tomar un string, una función, o una lista de estas, y calcular todas las agregaciones a la vez:

In [11]:
planets.groupby('method')[["distance", "mass"]].aggregate(['min', np.median, max])

/var/folders/50/q05bl7gj7jxb0qq6rb67fp4m0000gn/T/ipykernel_84888/1681637294.py:1: FutureWarning: The provided callable <function median at 0x107f9f5b0> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  planets.groupby('method')[["distance", "mass"]].aggregate(['min', np.median, max])
/var/folders/50/q05bl7gj7jxb0qq6rb67fp4m0000gn/T/ipykernel_84888/1681637294.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  planets.groupby('method')[["distance", "mass"]].aggregate(['min', np.median, max])


distance                       mass         \
                                   min    median      max     min median   
method                                                                     
Astrometry                       14.98    17.875    20.77     NaN    NaN   
Eclipse Timing Variations       130.72   315.360   500.00  4.2000  5.125   
Imaging                           7.69    40.395   165.00     NaN    NaN   
Microlensing                   1760.00  3840.000  7720.00     NaN    NaN   
Orbital Brightness Modulation  1180.00  1180.000  1180.00     NaN    NaN   
Pulsar Timing                  1200.00  1200.000  1200.00     NaN    NaN   
Pulsation Timing Variations        NaN       NaN      NaN     NaN    NaN   
Radial Velocity                   1.35    40.445   354.00  0.0036  1.260   
Transit                          38.00   341.000  8500.00  1.4700  1.470   
Transit Timing Variations       339.00   855.000  2119.00     NaN    NaN   

                                      
                                 max  
method                                
Astrometry                       NaN  
Eclipse Timing Variations       6.05  
Imaging                          NaN  
Microlensing                     NaN  
Orbital Brightness Modulation    NaN  
Pulsar Timing                    NaN  
Pulsation Timing Variations      NaN  
Radial Velocity                25.00  
Transit                         1.47  
Transit Timing Variations        NaN

Otra forma util es pasar un mapeado con un diccionario:

In [12]:
planets.groupby('method').aggregate({
    'distance': 'min',
    'mass': 'median'
})

,distance,mass
method,,
Astrometry,14.98,NaN
Eclipse Timing Variations,130.72,5.125
Imaging,7.69,NaN
Microlensing,1760.00,NaN
Orbital Brightness Modulation,1180.00,NaN
Pulsar Timing,1200.00,NaN
Pulsation Timing Variations,NaN,NaN
Radial Velocity,1.35,1.260
Transit,38.00,1.470


### filter

Una operación de filtrado te permite eliminar datos basados en las propiedades del grupo. Por ejemplo, podríamos querer mantener todos los grupos en los que la desviación estándar sea mayor que algún valor crítico:

In [13]:
def filter_func(x):
    return x['distance'].std() > 215

In [14]:
planets.groupby('method')[["distance"]].std()

,distance
method,
Astrometry,4.094148
Eclipse Timing Variations,213.203907
Imaging,53.736817
Microlensing,2076.611556
Orbital Brightness Modulation,0.000000
Pulsar Timing,NaN
Pulsation Timing Variations,NaN
Radial Velocity,45.559381
Transit,913.876990


In [15]:
planets.groupby('method').filter(filter_func).head()

,method,number,orbital_period,mass,distance,year
91,Transit,1,1.508956,NaN,NaN,2008
92,Transit,1,1.742994,NaN,200.0,2008
93,Transit,1,4.256800,NaN,680.0,2008
94,Transit,1,9.202050,NaN,NaN,2008
95,Transit,1,4.037896,NaN,NaN,2009


La función `filter_func` debe devolver un valor booleano que especifique si el grupo pasa el filtro. Acá, dado que los grupos `Astrometry`, `Imaging`, `Orbital Brightness Modulation`, `Pulsar Timing`, `Pulsation Timing Variations` y `Radial Velocity` no superan un desvio estándar de 215, se filtran.

----
### transform

Mientras que la agregación debe devolver una versión reducida de los datos, la transformación puede devolver una versión transformada de todos los datos para recombinarlos. Para dicha transformación, la salida tiene la misma forma que la entrada. Un ejemplo común es centrar los datos restando la media por grupo:

In [16]:
planets.groupby('method')[["distance"]].transform(lambda x: x - x.mean())

,distance
0,25.799792
1,5.349792
2,-31.760208
3,59.019792
4,67.869792
...,...
1030,-427.298080
1031,-451.298080
1032,-425.298080
1033,-306.298080


### apply

El método `apply()` permite aplicar una función arbitraria a los resultados del grupo. La función debe tomar un `DataFrame` y devolver un objeto de Pandas (por ejemplo, `DataFrame`, `Series`) o un escalar.

In [17]:
def norm_by_distance(x):
    x['orbital_period'] /= x['distance'].sum()
    return x

planets.groupby('method')[['orbital_period', "distance"]].apply(norm_by_distance)

orbital_period  distance
method                                                  
Astrometry                113         6.891189     20.77
                          537        28.419580     14.98
Eclipse Timing Variations 32          8.101852       NaN
                          37          4.571759    130.72
                          38          2.632705    130.72
...                                        ...       ...
Transit                   1034        0.000031    260.00
Transit Timing Variations 680         0.048295   2119.00
                          736         0.017208    855.00
                          749              NaN       NaN
                          813         0.006743    339.00

[1035 rows x 2 columns]